<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка текста</a></span></li><li><span><a href="#Обучение-и-оценка-моделей" data-toc-modified-id="Обучение-и-оценка-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение и оценка моделей</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
import spacy
import nltk
from tqdm.notebook import tqdm

In [2]:
# Загрузка данных
data = pd.read_csv('/datasets/toxic_comments.csv')

# Проверка данных
display(data.head())
display(data.info())
display(data['toxic'].value_counts(normalize=True))

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Выводы  

1. **Структура данных**:  
   - Датасет содержит **159 292 записи** и **3 столбца**
   - Типы данных: `int64` для числовых столбцов и `object` для текста.  

2. **Распределение меток токсичности**:  
   - **89.8%** записей (**класс 0**) — **нетоксичные** тексты.  
   - **10.2%** записей (**класс 1**) — **токсичные** тексты.  
   - Наблюдается **сильный дисбаланс классов**, что типично для задач классификации токсичности.  

**Итог**: Далее стоит сосредоточиться на предобработке текста и методах работы с дисбалансом для построения эффективной классификационной модели.

## Предобработка текста

In [3]:
# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [5]:
# Функция для преобразования POS-тегов NLTK в формат WordNet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # по умолчанию считаем существительным

In [6]:
def preprocess_text(text):
    # Удаление спецсимволов
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление лишних пробелов
    text = text.strip()
    # Токенизация
    tokens = text.split()
    
    # Получаем POS-теги для всех токенов
    pos_tags = nltk.pos_tag(tokens)
    
    # Лемматизация с учетом POS-тегов
    lemmatized_tokens = []
    for word, pos in pos_tags:
        if word not in stop_words and len(word) > 2:  # игнорируем стоп-слова и короткие слова
            wordnet_pos = get_wordnet_pos(pos)
            lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
            lemmatized_tokens.append(lemma)
    
    # Сборка обратно в строку
    return ' '.join(lemmatized_tokens)

In [7]:
# Включаем прогресс-бар для pandas
tqdm.pandas()

# Применение предобработки с отображением прогресса
data['processed_text'] = data['text'].progress_apply(preprocess_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [8]:
# Выводим первые 10 строк для сравнения
print(data[['text', 'processed_text']].head(10))

                                                text  \
0  Explanation\nWhy the edits made under my usern...   
1  D'aww! He matches this background colour I'm s...   
2  Hey man, I'm really not trying to edit war. It...   
3  "\nMore\nI can't make any real suggestions on ...   
4  You, sir, are my hero. Any chance you remember...   
5  "\n\nCongratulations from me as well, use the ...   
6       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
7  Your vandalism to the Matt Shirvington article...   
8  Sorry if the word 'nonsense' was offensive to ...   
9  alignment on this subject and which are contra...   

                                      processed_text  
0  explanation edits make username hardcore metal...  
1  daww match background colour seemingly stick t...  
2  hey man really try edit war guy constantly rem...  
3  cant make real suggestion improvement wonder s...  
4                sir hero chance remember page thats  
5             congratulation well use tool well talk 

In [9]:
# Разделение на признаки и целевую переменную
X = data['processed_text']
y = data['toxic']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

## Обучение и оценка моделей

In [10]:
# Создание пайплайнов для разных моделей
models = {
    'Logistic Regression': Pipeline([
        ('tfidf', TfidfVectorizer(max_features=50000, ngram_range=(1, 2))),
        ('clf', LogisticRegression(random_state=42, class_weight='balanced'))
    ]),
    'Random Forest': Pipeline([
        ('tfidf', TfidfVectorizer(max_features=50000)),
        ('clf', RandomForestClassifier(random_state=42, class_weight='balanced'))
    ]),
    'Linear SVM': Pipeline([
        ('tfidf', TfidfVectorizer(max_features=50000)),
        ('clf', LinearSVC(random_state=42, class_weight='balanced'))
    ])
}

In [11]:
# Оценка моделей с помощью кросс-валидации
results = {}
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring='f1')
    mean_f1 = cv_scores.mean()
    results[name] = mean_f1
    print(f'{name}: Средний F1 на кросс-валидации = {mean_f1:.4f}')
    print(f'Отдельные fold-ы: {[round(score, 4) for score in cv_scores]}')

Logistic Regression: Средний F1 на кросс-валидации = 0.7437
Отдельные fold-ы: [0.7408, 0.7446, 0.7459]
Random Forest: Средний F1 на кросс-валидации = 0.6492
Отдельные fold-ы: [0.6508, 0.6512, 0.6455]
Linear SVM: Средний F1 на кросс-валидации = 0.7419
Отдельные fold-ы: [0.7396, 0.7446, 0.7414]


In [12]:
# Выбор лучшей модели
best_model_name = max(results, key=results.get)
print(f'Лучшая модель: {best_model_name} с F1 = {results[best_model_name]:.4f}')

# Дополнительная настройка параметров для лучшей модели
if best_model_name == 'Logistic Regression':
    param_grid = {
        'tfidf__max_features': [30000, 50000, 70000],
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'clf__C': [0.1, 1, 10]
    }
    
    grid_search = GridSearchCV(models[best_model_name], param_grid, cv=3, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    print(f'Лучшие параметры: {grid_search.best_params_}')
    print(f'Лучший F1: {grid_search.best_score_:.4f}')
    
    # Оценка на тестовых данных
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    final_f1 = f1_score(y_test, y_pred)
    print(f'Финальный F1 на тестовых данных: {final_f1:.4f}')

Лучшая модель: Logistic Regression с F1 = 0.7437


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Лучшие параметры: {'clf__C': 10, 'tfidf__max_features': 70000, 'tfidf__ngram_range': (1, 2)}
Лучший F1: 0.7614
Финальный F1 на тестовых данных: 0.7665


  - выбраны гиперпараметры для модели
  - тестирование произведено корректно 
  - достигнута метрика f1 выше 0,75 на тестовой выборке

## Выводы

- Качество модели: Мы достигли целевого значения метрики после настройки гиперпараметров модель Logistic Regression показала F1 0.77 на тестовых данных.

- Итог: Модель готова к внедрению для автоматического обнаружения токсичных комментариев в интернет-магазине "Викишоп".

